# Initialize workspace

In [ ]:
library(here)

In [ ]:
PACKAGES <- rlang::quos(
    arrow,
    car,
    cardx,
    haven,
    lubridate,
    broom,
    fastDummies,
    fs,
    ggpubr,
    gt,
    gtsummary,
    skimr,
    tidymodels,
    tidyverse)

lapply(PACKAGES, rlang::quo_name) |> lapply(library, character.only = TRUE) |>
    invisible()

In [ ]:
ROOT <- path(
    'data', 
    'raw')

In [ ]:
ROOT_INTERMEDIATE <- path(
    'data',
    'intermediate')

# Import data and set up dataframes

## Baseline

### Get components

#### Characteristics

In [ ]:
df_characteristics <- read_sas(here(ROOT,'baseline.sas7bdat')) |>
    rename(
        randomized = RANDOMED,
        randomization_date = RANDODT,
        age_stratum = AGESTRAT,
        function_stratum = FUNSTRAT,
        randomization_id = RANDOID,
        age_years = AGEYRS,
        age_months = AGEMONS,
        male_sex = MALE,
        latin_ethnicity = HISPANIC,
        race = RACE_D,
        family_status = FAMSTAT,
        childbearing = CHDBR,
        tx_group = TX,
        site_id = SITE_R)

#### IQ

In [ ]:
df_iq <- read_sas(here(ROOT,'iq_test.sas7bdat')) |>
    rename(
        baseline_iq_sb = SB5ABIQ,
        baseline_iq_mullen = MULLENIQ,
        visit_raw = visit,
        visit = VISIT_D) |>
        filter(visit==-1) |>
        mutate(baseline_iq = coalesce(baseline_iq_sb, baseline_iq_mullen)) |>
        select(subject, baseline_iq)

#### BMI

In [ ]:
df_bmi <- read_sas(here(ROOT,'vitals.sas7bdat')) |>
    rename(
        baseline_bmi_z_score = Z_BMI_B)

df_bmi <- df_bmi |> select(subject, baseline_bmi_z_score)

#### Medications

In [ ]:
df_meds <- read_sas(here(ROOT,'conmed.sas7bdat')) |>
    filter(visit==0) |>
    rename(med_group = CM_GRP)

In [ ]:
df_meds <- read_sas(here(ROOT,'conmed.sas7bdat')) |>
    filter(visit==0) |>
    rename(med_group = CM_GRP) |>
    dummy_cols(
        select_columns = "med_group",
        remove_first_dummy = FALSE) |>
    select(-c(visit, cm_dose, cm_seq, cm_code, med_group, FUNSTRAT, TX, med_group_)) |>
    group_by(subject) |> summarize(
        med_group_A = max(med_group_A),
        med_group_B = max(med_group_B),
        med_group_D = max(med_group_D),
        med_group_E = max(med_group_E),
        med_group_F = max(med_group_F),
        med_group_G = max(med_group_G),
        med_group_H = max(med_group_H),
        med_group_I = max(med_group_I),
        med_group_J = max(med_group_J),
        med_group_L = max(med_group_L),
        med_group_M = max(med_group_M),
        med_group_O = max(med_group_O),
        med_group_P = max(med_group_P),
        med_group_Q = max(med_group_Q)
) |>
    rename(
        med_alpha_agonist = med_group_A,
        med_anxiolytic = med_group_B,
        med_anticonvulsant = med_group_D,
        med_antipsychotic = med_group_E,
        med_stimulant = med_group_F,
        med_sleep = med_group_G,
        med_gi = med_group_H,
        med_infection = med_group_I,
        med_allergy = med_group_J,
        med_steroid_inh = med_group_L,
        med_supplement = med_group_M,
        med_other = med_group_O,
        med_antidepressant = med_group_P,
        med_nonstimulant_adhd = med_group_Q)

#### Baseline severity

In [ ]:
df_cgi <- read_sas(here(ROOT,'cgi.sas7bdat')) |>
    rename(
        baseline_cgi_severity = CGI_SI0,
        cgi_severity = CGI_SI,
        cgi_improvement = CGI_GI,
        visit_raw = visit,
        visit = VISIT_D) |>
    filter(visit==0) |>
    mutate(baseline_high_severity = if_else(cgi_severity >= 5, 1, 0)) |>
    select(subject, baseline_high_severity)

#### Baseline ABC

In [ ]:
df_abc <- read_sas(here(ROOT,'abc.sas7bdat')) |>
    rename(
        subject = SUBJECT,
        visit = VISIT_D,
        abc_sw_baseline = abc_sw0,
        abc_speech_baseline = abc_s5b,
        abc_hyperactivity_baseline = abc_s4b,
        abc_stereotypy_baseline = abc_s3b,
        abc_lethargy_baseline = abc_s2b,
        abc_irritability_baseline = abc_s1b
    )
df_abc_baseline <- df_abc |> filter(visit==0) |>
    select(
        subject, 
        abc_sw_baseline,
        abc_speech_baseline,
        abc_hyperactivity_baseline,
        abc_stereotypy_baseline,
        abc_lethargy_baseline,
        abc_irritability_baseline)

#### Baseline Vineland

In [ ]:
df_vineland <- read_sas(here(ROOT,'vineland.sas7bdat')) |>
    rename(
        visit_raw = visit,
        visit = VISIT_D,
        vineland_socialization_standard = SOCIA_SS,
        vineland_communication_standard = COMNI_SS)

df_vineland_baseline <- df_vineland |>
    filter(visit==0) |>
    select(
        subject,
        vineland_socialization_standard,
        vineland_communication_standard) |>
    mutate(
        vineland_2dc = (vineland_socialization_standard + vineland_communication_standard)/2) |>
    rename(
        vineland_socialization_standard_baseline = vineland_socialization_standard,
        vineland_communication_standard_baseline = vineland_communication_standard,
        vineland_2dc_baseline = vineland_2dc)

#### Baseline caregiver strain

In [ ]:
df_csq_baseline <- read_sas(here(ROOT,'csq.sas7bdat')) |>
    rename(
        caregiver_strain_objective_baseline = CSQ_OS0,
        caregiver_strain_subjective_internal_baseline = CSQ_SIS0,
        caregiver_strain_subjective_external_baseline = CSQ_SES0,
        visit_raw = visit,
        visit = VISIT_D) |>
    filter(visit==0) |>
    select(subject,
           caregiver_strain_objective_baseline,
           caregiver_strain_subjective_internal_baseline,
           caregiver_strain_subjective_external_baseline)

### Merge

In [ ]:
df_baseline <- list(
    df_characteristics,
    df_bmi,
    df_iq,
    df_meds,
    df_cgi,
    df_abc_baseline,
    df_vineland_baseline,
    df_csq_baseline) |>
reduce(
    left_join, by = "subject")

In [ ]:
df_baseline |> nrow()

## Outcomes

### Get components

#### SRS

In [ ]:
df_srs <- read_sas(here(ROOT,'srs.sas7bdat')) |>
    rename(
        subject = SUBJECT,
        srs_raw_total = SRS_TOT,
        visit = VISIT_D) 

df_srs_baseline <- df_srs |>
    filter(visit==0) |>
    rename(
        srs_raw_total_baseline = srs_raw_total) |>
    select(subject, srs_raw_total_baseline)

In [ ]:
df_srs_week_12 <- df_srs |> filter(visit==12) |>
    select(subject, srs_raw_total) |>
    rename(srs_raw_total_12 = srs_raw_total)

df_srs_week_24 <- df_srs |> filter(visit==24) |>
    select(subject, srs_raw_total) |>
    rename(srs_raw_total_24 = srs_raw_total)

In [ ]:
df_srs_week_12 <- df_srs_baseline |> left_join(df_srs_week_12, by = "subject") |>
    mutate(srs_change_12 = srs_raw_total_baseline - srs_raw_total_12) |>
    select(subject, srs_change_12, srs_raw_total_baseline)

In [ ]:
df_srs_week_24 <- df_srs_baseline |> left_join(df_srs_week_24, by = "subject") |>
    mutate(srs_change_24 = srs_raw_total_baseline - srs_raw_total_24) |>
    select(subject, srs_change_24, srs_raw_total_baseline)

In [ ]:
df_srs_change <- df_srs_week_12 |> left_join(df_srs_week_24)

#### CGI-Improvement

In [ ]:
df_redcap <- read_sas(here(ROOT, 'redcap_nonslaes.sas7bdat'))
df_redcap_12 <- df_redcap |> filter(visit==12)
df_redcap_24 <- df_redcap |> filter(visit==24)


In [ ]:
df_cgi_12_redcap <- df_redcap_12 |> select(subject, cgigi)

In [ ]:
df_cgi_24_redcap <- df_redcap_24 |> select(subject, cgigi)


In [ ]:
df_cgi <- read_sas(here(ROOT,'cgi.sas7bdat')) |>
    rename(
        cgi_severity_baseline = CGI_SI0,
        cgi_severity = CGI_SI,
        cgi_improvement = CGI_GI,
        visit_raw = visit,
        visit = VISIT_D) 

In [ ]:
df_cgi_improvement <- read_sas(here(ROOT,'cgi.sas7bdat')) |>
    rename(
        cgi_severity_baseline = CGI_SI0,
        cgi_severity = CGI_SI,
        cgi_improvement = CGI_GI,
        visit_raw = visit,
        visit = VISIT_D) 

In [ ]:
df_cgi_improvement_week_12 <- df_cgi_improvement |>
    filter(visit==12) |>
    select(subject, cgi_improvement, cgi_severity_baseline) |>
    rename(cgi_improvement_wk_12 = cgi_improvement)

In [ ]:
df_cgi_improvement_week_12 <- df_cgi_improvement_week_12 |>
  left_join(df_cgi_12_redcap, by = "subject") |>
  mutate(cgi_improvement_wk_12 = ifelse(is.na(cgi_improvement_wk_12), cgigi, cgi_improvement_wk_12)) |>
  select(-cgigi)

In [ ]:
df_cgi_improvement_week_24 <- df_cgi_improvement |>
    filter(visit==24) |>
    select(subject, cgi_improvement, cgi_severity_baseline) |>
    rename(cgi_improvement_wk_24 = cgi_improvement)

In [ ]:
df_cgi_improvement_week_24 <- df_cgi_improvement_week_24 |>
    left_join(df_cgi_24_redcap, by = "subject") |>
    mutate(cgi_improvement_wk_24 = ifelse(is.na(cgi_improvement_wk_24), cgigi, cgi_improvement_wk_24)) |>
    select(-cgigi)

In [ ]:
df_cgi_improvement <- df_cgi_improvement_week_12 |> left_join(df_cgi_improvement_week_24)

#### ABC-SW Change

In [ ]:
df_abc_week_12 <- df_abc |> filter(visit==12) |>
    select(subject, abc_sw) |>
    rename(abc_sw_12 = abc_sw)

In [ ]:
df_abc_week_24 <- df_abc |> filter(visit==24) |>
    select(subject, abc_sw) |>
    rename(abc_sw_24 = abc_sw)

In [ ]:
df_abc_week_12 <- df_abc_baseline |> left_join(df_abc_week_12, by = "subject") |>
    mutate(abc_sw_change_12 = abc_sw_baseline - abc_sw_12) |>
    select(subject, abc_sw_change_12, abc_sw_baseline)

In [ ]:
df_abc_week_24 <- df_abc_baseline |> left_join(df_abc_week_24, by = "subject") |>
    mutate(abc_sw_change_24 = abc_sw_baseline - abc_sw_24) |>
    select(subject, abc_sw_change_24, abc_sw_baseline)

In [ ]:
df_abc <- df_abc_week_12 |> left_join(df_abc_week_24)

In [ ]:
df_abc <- df_abc |> select(-abc_sw_baseline)

### Merge

In [ ]:
df_outcomes <- list(
    df_cgi_improvement,
    df_abc,
    df_srs_change
) |>
reduce(
    left_join, by = "subject")

## Final merge

In [ ]:
df <- list(
    df_baseline,
    df_outcomes
) |>
reduce(
    left_join, by = "subject")

In [ ]:
df <- df |> mutate(
    cgi_improved_at_12 = if_else(cgi_improvement_wk_12 <= 2, 1, 0))

In [ ]:
df <- df |> mutate(
    cgi_improved_at_24 = if_else(cgi_improvement_wk_24 <= 2, 1, 0))

In [ ]:
df |> write_feather(here(ROOT_INTERMEDIATE, 'df.feather'))

In [ ]:
df_freeze <- df

In [ ]:
# account for missingness that can be accounted for
df <- df |>
    mutate(race = replace_na(race, 1)) |>
    mutate(latin_ethnicity = replace_na(latin_ethnicity, 0)) |>
    mutate(baseline_high_severity = replace_na(baseline_high_severity, 0)) |>
    mutate(med_alpha_agonist = replace_na(med_alpha_agonist, 0)) |> 
    mutate(med_anxiolytic = replace_na(med_anxiolytic, 0)) |> 
    mutate(med_anticonvulsant = replace_na(med_anticonvulsant, 0)) |> 
    mutate(med_antipsychotic = replace_na(med_antipsychotic, 0)) |> 
    mutate(med_stimulant = replace_na(med_stimulant, 0)) |> 
    mutate(med_sleep = replace_na(med_sleep, 0)) |> 
    mutate(med_gi = replace_na(med_gi, 0)) |> 
    mutate(med_infection = replace_na(med_infection, 0)) |>
    mutate(med_allergy = replace_na(med_allergy, 0)) |> 
    mutate(med_steroid_inh = replace_na(med_steroid_inh, 0)) |> 
    mutate(med_supplement = replace_na(med_supplement, 0)) |>
    mutate(med_other = replace_na(med_other, 0)) |> 
    mutate(med_antidepressant = replace_na(med_antidepressant, 0)) |>
    mutate(med_nonstimulant_adhd = replace_na(med_nonstimulant_adhd, 0))

In [ ]:
df <- df |> 
    select(-c(
        subject,
        strata,
        baseline_high_severity,
        randomized,
        randomization_date,
        base_dt,
        age_stratum,
        function_stratum,
        randomization_id,
        age_years,
        family_status,
        childbearing))

In [ ]:
df <- df |> distinct()

In [ ]:
df$site_id <- as.factor(df$site_id)

In [ ]:
df$race <- as.factor(df$race)

In [ ]:
df <- df |> dummy_cols(select_columns = "race", remove_first_dummy = FALSE) |>
    rename(
        race_white = race_1,
        race_black = race_2,
        race_asian = race_3,
        race_more_than_one = race_9)

In [ ]:
df <- df |> dummy_cols(select_columns = "site_id", remove_first_dummy = FALSE)

In [ ]:
df_tx <- df |> filter(tx_group==1)

df_placebo <- df |> filter(tx_group==0)

# Descriptives

In [ ]:
df |> group_by(tx_group) |>
    summarize(
        responded = sum(cgi_improved_at_12 == 1, na.rm = TRUE),
        total = sum(!is.na(cgi_improved_at_12)),
        raw_total = n(),
        response_rate = (responded / total) * 100)

In [ ]:
df_table <- df

In [ ]:
df_table <- df_table |> mutate(
    age_years = age_months/12)

In [ ]:
df_table$race <- recode_factor(df_table$race,
    `1` = "White",
    `9` = "More than one race",
    `2` = "Black",
    `3` = "Asian")

In [ ]:
df_table$male_sex <- as.factor(df_table$male_sex)

In [ ]:
df_table$male_sex <- recode_factor(df_table$male_sex,
    `1` = "Male",
    `0` = "Female")

In [ ]:
df_table$latin_ethnicity <- recode_factor(df_table$latin_ethnicity,
    `1` = "Latine ethnicity",
    `0` = "Non-Latine ethnicity")

In [ ]:
df_table$latin_ethnicity <- recode_factor(df_table$latin_ethnicity,
    `1` = "Latine ethnicity",
    `0` = "Non-Latine ethnicity")

In [ ]:
df_table$tx_group <- recode_factor(df_table$tx_group,
    `0` = "Placebo",
    `1` = "Treatment")

In [ ]:
df_table <- df_table |>
    select(
        age_years,
        site_id,
        male_sex,
        race,
        latin_ethnicity,
        tx_group,
        abc_sw_baseline,
        abc_speech_baseline,
        abc_hyperactivity_baseline,
        abc_stereotypy_baseline,
        abc_lethargy_baseline,
        abc_irritability_baseline,
        vineland_socialization_standard_baseline,
        vineland_communication_standard_baseline,
        vineland_2dc_baseline,
        caregiver_strain_objective_baseline,
        caregiver_strain_subjective_internal_baseline,
        caregiver_strain_subjective_external_baseline,
        cgi_severity_baseline,
        srs_raw_total_baseline,
        baseline_iq,
        baseline_bmi_z_score,
        med_alpha_agonist,
        med_anxiolytic,
        med_antidepressant,
        med_anticonvulsant,
        med_antipsychotic,
        med_stimulant,
        med_sleep,
        med_gi,
        abc_sw_change_12,
        abc_sw_change_24,
        cgi_improvement_wk_12,
        cgi_improvement_wk_24,
        cgi_improved_at_12,
        cgi_improved_at_24
    )

In [ ]:
table1 <- df_table |> tbl_summary(
    by = tx_group,
    type = list(
        age_years ~ "continuous",
        abc_sw_baseline ~ "continuous",
        abc_speech_baseline ~ "continuous",
        abc_hyperactivity_baseline ~ "continuous",
        abc_stereotypy_baseline ~ "continuous",
        abc_lethargy_baseline ~ "continuous",
        abc_irritability_baseline ~ "continuous",
        vineland_socialization_standard_baseline ~ "continuous",
        vineland_communication_standard_baseline ~ "continuous",
        vineland_2dc_baseline ~ "continuous",
        caregiver_strain_objective_baseline ~ "continuous",
        caregiver_strain_subjective_internal_baseline ~ "continuous",
        caregiver_strain_subjective_external_baseline ~ "continuous",
        cgi_severity_baseline ~ "continuous",
        srs_raw_total_baseline ~ "continuous",
        abc_sw_change_12 ~ "continuous",
        abc_sw_change_24 ~ "continuous",
        cgi_improvement_wk_12 ~ "continuous",
        cgi_improvement_wk_24 ~ "continuous"),
    statistic = all_continuous() ~ "{mean} ({sd})",
    digits = all_continuous() ~ 2,
    include = c(
        age_years,
        male_sex,
        race,
        latin_ethnicity,
        site_id,
        abc_sw_baseline,
        abc_lethargy_baseline,
        abc_speech_baseline,
        abc_hyperactivity_baseline,
        abc_stereotypy_baseline,
        abc_irritability_baseline,
        vineland_socialization_standard_baseline,
        vineland_communication_standard_baseline,
        vineland_2dc_baseline,
        caregiver_strain_objective_baseline,
        caregiver_strain_subjective_internal_baseline,
        caregiver_strain_subjective_external_baseline,
        cgi_severity_baseline,
        srs_raw_total_baseline,
        baseline_iq,
        baseline_bmi_z_score,
        med_alpha_agonist,
        med_anxiolytic,
        med_antidepressant,
        med_anticonvulsant,
        med_antipsychotic,
        med_stimulant,
        med_sleep,
        med_gi,
        abc_sw_change_12,
        abc_sw_change_24,
        cgi_improvement_wk_12,
        cgi_improvement_wk_24,
        cgi_improved_at_12,
        cgi_improved_at_24),
    label = list(
        age_years ~ "Age in years",
        site_id ~ "Study site",
        male_sex ~ "Sex at birth",
        race ~ "Race",
        latin_ethnicity ~ "Ethnicity",
        abc_sw_baseline ~ "Baseline ABC-modified Social Withdrawal",
        abc_speech_baseline ~ "Baseline ABC-Speech",
        abc_hyperactivity_baseline ~ "Baseline ABC-Hyperactivity",
        abc_stereotypy_baseline ~ "Baseline ABC-Stereotypy",
        abc_lethargy_baseline ~ "Baseline ABC-modified Lethargy",
        abc_irritability_baseline ~ "Baseline ABC-Irritability",
        vineland_socialization_standard_baseline ~ "Baseline VABS-II Socialization",
        vineland_communication_standard_baseline ~ "Baseline VABS-II Communication",
        vineland_2dc_baseline ~ "Baseline VABS-II 2DC",
        caregiver_strain_objective_baseline ~ "Baseline CSQ Objective",
        caregiver_strain_subjective_internal_baseline ~ "Baseline CSQ Subjective Internal",
        caregiver_strain_subjective_external_baseline ~ "Baseline CSQ Subjective External",
        cgi_severity_baseline ~ "CGI-S",
        srs_raw_total_baseline ~ "SRS-2",
        baseline_iq ~ "SB5 Abbreviated IQ",
        baseline_bmi_z_score ~ "BMI z-score",
        med_alpha_agonist ~ "Alpha agonist",
        med_anxiolytic ~ "Anxiolytic",
        med_antidepressant ~ "Antidepressant",
        med_anticonvulsant ~ "Anticonvulsant",
        med_antipsychotic ~ "Antipsychotic",
        med_stimulant ~ "Stimulant",
        med_sleep ~ "Sleep aid",
        med_gi ~ "GI medication", 
        abc_sw_change_12 ~ "12-week change in ABC-mSW",
        abc_sw_change_24 ~ "24-week change in ABC-mSW",
        cgi_improvement_wk_12 ~ "12-week CGI-I",
        cgi_improvement_wk_24 ~ "24-week CGI-I",
        cgi_improved_at_12 ~ "12-week response",
        cgi_improved_at_24 ~ "24-week response"),
    missing = "no") |>
    add_overall(last = FALSE) |>
    modify_column_alignment(columns = all_stat_cols(), align = "right") |>
    modify_spanning_header(
        all_stat_cols() ~ "**Comparison of placebo and treatment groups**") |>
    add_p(pvalue_fun = ~ style_pvalue(.x, digits = 2)) |>
    as_gt() |>
    gtsave(here("output","table1.html"))        

# LASSO

## Placebo group

### 24 weeks

#### ABC-mSW Change

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    abc_sw_change_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) |> bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe |> prep(data = df_test) |> bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_placebo_abc_24 <- best_model

In [ ]:
tidy(lasso_placebo_abc_24)

In [ ]:
tidy(lasso_placebo_abc_24)
tidy(lasso_placebo_abc_24) |> write_csv(here("output","lasso_placebo_abc_24.csv"))        

In [ ]:
predictors_of_outcome <- tidy(lasso_placebo_abc_24) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at 24 weeks\n in the placebo group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-mSW"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

In [ ]:
ggsave(here("output","graph_lasso_placebo_abc_24.png"))

In [ ]:
ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "Baseline ABC-mSW vs. change at 24 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 24 weeks") +
    theme_minimal()


In [ ]:
ggsave(here("output","graph_unpenalized_placebo_24.png"))

In [ ]:
ggplot(df_tx, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "ABC-mSW: baseline vs. change at 24 weeks (unpenalized regression)\nin treatment group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW score at 24 weeks")


#### CGI Improvement

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    cgi_improvement_wk_24,

    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 24 weeks",
       x = "predictor",
       y = "importance")


### 12 weeks

#### ABC-SW Change

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    abc_sw_change_12,

    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_duke,
    site_mt_sinai,
    site_mgh,
    site_vanderbilt,
    site_seattle,
    site_cornell,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_placebo_abc_12 <- best_model

In [ ]:
tidy(lasso_placebo_abc_12)
tidy(lasso_placebo_abc_12) |> write_csv(here("output","lasso_placebo_abc_12.csv"))        

In [ ]:
predictors_of_outcome <- tidy(lasso_placebo_abc_12) |>
    filter(term!="(Intercept)")

In [ ]:
# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_duke" = "Study site 1",
    "site_mt_sinai" = "Study site 2",
    "site_mgh" = "Study site 3",
    "site_vanderbilt" = "Study site 4",
    "site_seattle" = "Study site 5",
    "site_cornell" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")


# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
graph_lasso_placebo_12 <- ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at 12 weeks\n in the placebo group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-mSW"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

In [ ]:
graph_lasso_placebo_12

In [ ]:
ggsave(here("output","graph_lasso_placebo_abc_12.png"))

In [ ]:
placebo_12_plot <- placebo_12_plot + theme(aspect.ratio = 1)

In [ ]:
combined_plot <- graph_lasso_placebo_24 + placebo_12_plot +
  plot_annotation(tag_levels = "a",  # Automatically label plots as 'A', 'B', etc.
                  caption = "Positive predictors are blue; negative predictors are red.\nPredictors that remained statistically significant after the verification step are marked with an asterisk.")  # Add a caption


In [ ]:
combined_plot

In [ ]:
ggsave(here("output","test.png"))

In [ ]:
placebo_12_plot

In [ ]:
graph_lasso_placebo_24

In [ ]:
tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC SW Change at 12 weeks",
       x = "predictor",
       y = "importance")

In [ ]:
# fit lasso line on untransformed data

# Step 1: Extract scaling and centering parameters from the recipe
prepped_recipe <- recipe %>% prep(data = df_train)
scaling_params <- tidy(prepped_recipe, number = 1)  # step_scale
centering_params <- tidy(prepped_recipe, number = 2)  # step_center

# Get the scaling (means and sds) for the variable of interest (abc_sw_baseline)
abc_sw_baseline_mean <- centering_params$value[centering_params$terms == "abc_sw_baseline"]
abc_sw_baseline_sd <- scaling_params$value[scaling_params$terms == "abc_sw_baseline"]

# Step 2: Extract LASSO coefficients for the best model
lasso_coeffs <- coef(best_model$fit, s = best_model$spec$args$penalty)

# Extract the coefficient for abc_sw_baseline
abc_sw_baseline_coeff <- lasso_coeffs[rownames(lasso_coeffs) == "abc_sw_baseline", 1]
intercept <- lasso_coeffs[1, 1]  # This is the intercept

# Step 3: Unscale the coefficient and intercept
unscaled_coeff <- abc_sw_baseline_coeff / abc_sw_baseline_sd
unscaled_intercept <- intercept - (abc_sw_baseline_mean * unscaled_coeff)

# Step 4: Use unscaled coefficients to predict on original data
df_test$predicted_change <- unscaled_intercept + df_test$abc_sw_baseline * unscaled_coeff

# Step 5: Plot untransformed data and LASSO line of best fit
ggplot(df_test, aes(x = abc_sw_baseline, y = abc_sw_change_12)) +
  geom_point(alpha = 0.5) +  # Plot actual data
  geom_line(aes(y = predicted_change), color = "cornflowerblue", size = 1) +  # LASSO line of best fit
  labs(title = "ABC-SW: baseline vs change at 12 weeks (LASSO)",
       x = "Baseline ABC-SW score",
       y = "Change in ABC-SW score at 12 weeks")


In [ ]:
ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  stat_regline_equation(label.x = 5, label.y = 20, aes(label = paste(after_stat(eq.label)))) + # Adds regression equation with customized position
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "Baseline ABC-mSW vs. change at 24 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 24 weeks") +
    theme_minimal()


In [ ]:
ggsave("line1.png")

In [ ]:
ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "Baseline ABC-mSW vs. change at 24 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 24 weeks") +
    theme_minimal()


In [ ]:
ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_12)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  stat_regline_equation(label.x = 5, label.y = 20, aes(label = paste(after_stat(eq.label)))) + # Adds regression equation with customized position
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
labs(title = "Baseline ABC-mSW vs. change at 12 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 12 weeks") +
    theme_minimal()

In [ ]:
ggsave("line2.png")

In [ ]:
ggsave(here("output", "graph_unpenalized_placebo_12.png"))

#### CGI Improvement

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    cgi_improvement_wk_12,
    
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_duke,
    site_mt_sinai,
    site_mgh,
    site_vanderbilt,
    site_seattle,
    site_cornell,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 12 weeks",
       x = "predictor",
       y = "importance")


In [ ]:
# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_duke" = "Study site 1",
    "site_mt_sinai" = "Study site 2",
    "site_mgh" = "Study site 3",
    "site_vanderbilt" = "Study site 4",
    "site_seattle" = "Study site 5",
    "site_cornell" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")


# Replace terms with new names in the data
predictors_of_cgi_improvement <- predictors_of_cgi_improvement |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI-I at 12 weeks\n in the placebo group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Age"), 
            aes(label = "*"), hjust = -3, color = "black", size = 6, fontface = "bold") + # Adjust hjust to position the asterisk +
geom_text(data = filter(predictors_of_outcome, term == "SB5 Abbreviated IQ"), 
            aes(label = "*"), hjust = -0.6, color = "black", size = 6, fontface = "bold") +
geom_text(data = filter(predictors_of_outcome, term == "Study site 1"), 
            aes(label = "*"), hjust = 30.5, color = "black", size = 6, fontface = "bold")

In [ ]:
ggsave(here("output","graph_lasso_placebo_cgi_12.png"))

In [ ]:
df_placebo |> colnames()

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    abc_sw_change_12,
    age_months,
    male_sex,
    latin_ethnicity,
    race,
    site_id,
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    baseline_bmi_z_score,
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,
    med_nonstimulant_adhd,
    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 12 weeks",
       x = "predictor",
       y = "importance")


## Treatment group

### 24 weeks

#### ABC-SW Change

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    abc_sw_change_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_duke,
    site_mt_sinai,
    site_mgh,
    site_vanderbilt,
    site_seattle,
    site_cornell,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
tidy(best_model)

lasso_12 <- best_model

predictors_of_outcome <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-SW Change at 24 weeks",
       x = "predictor",
       y = "importance")

In [ ]:
# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_duke" = "Study site 1",
    "site_mt_sinai" = "Study site 2",
    "site_mgh" = "Study site 3",
    "site_vanderbilt" = "Study site 4",
    "site_seattle" = "Study site 5",
    "site_cornell" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at 24 weeks\n in the oxytocin group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-mSW"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold") + # Adjust hjust to position the asterisk +
geom_text(data = filter(predictors_of_outcome, term == "SRS-2"), 
            aes(label = "*"), hjust = 1.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

In [ ]:
ggsave(here("output","graph_lasso_oxytocin_abc_24.png"))

In [ ]:
# fit lasso line on untransformed data

# Step 1: Extract scaling and centering parameters from the recipe
prepped_recipe <- recipe %>% prep(data = df_train)
scaling_params <- tidy(prepped_recipe, number = 1)  # step_scale
centering_params <- tidy(prepped_recipe, number = 2)  # step_center

# Get the scaling (means and sds) for the variable of interest (abc_sw_baseline)
abc_sw_baseline_mean <- centering_params$value[centering_params$terms == "abc_sw_baseline"]
abc_sw_baseline_sd <- scaling_params$value[scaling_params$terms == "abc_sw_baseline"]

# Step 2: Extract LASSO coefficients for the best model
lasso_coeffs <- coef(best_model$fit, s = best_model$spec$args$penalty)

# Extract the coefficient for abc_sw_baseline
abc_sw_baseline_coeff <- lasso_coeffs[rownames(lasso_coeffs) == "abc_sw_baseline", 1]
intercept <- lasso_coeffs[1, 1]  # This is the intercept

# Step 3: Unscale the coefficient and intercept
unscaled_coeff <- abc_sw_baseline_coeff / abc_sw_baseline_sd
unscaled_intercept <- intercept - (abc_sw_baseline_mean * unscaled_coeff)

# Step 4: Use unscaled coefficients to predict on original data
df_test$predicted_change <- unscaled_intercept + df_test$abc_sw_baseline * unscaled_coeff

# Step 5: Plot untransformed data and LASSO line of best fit
ggplot(df_test, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Plot actual data
  geom_line(aes(y = predicted_change), color = "cornflowerblue", size = 1) +  # LASSO line of best fit
  labs(title = "ABC-SW: baseline vs change at 24 weeks (LASSO) in treatment group",
       x = "Baseline ABC-SW score",
       y = "Change in ABC-SW score at 24 weeks")


In [ ]:
ggplot(df_tx, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "lightcoral") + # unpenalized regression line
  labs(title = "ABC-SW: baseline vs change at 24 weeks (unpenalized regression)\n in treatment group",
       x = "Baseline ABC-SW score",
       y = "Change in ABC-SW score at 24 weeks")

#### CGI Improvement

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    cgi_improvement_wk_24,

    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_duke,
    site_mt_sinai,
    site_mgh,
    site_vanderbilt,
    site_seattle,
    site_cornell,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 24 weeks",
       x = "predictor",
       y = "importance")


In [ ]:
# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_duke" = "Study site 1",
    "site_mt_sinai" = "Study site 2",
    "site_mgh" = "Study site 3",
    "site_vanderbilt" = "Study site 4",
    "site_seattle" = "Study site 5",
    "site_cornell" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")

# Replace terms with new names in the data
predictors_of_cgi_improvement <- predictors_of_cgi_improvement |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI-I at 24 weeks\n in the oxytocin group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Male sex"), 
            aes(label = "*"), hjust = -11, color = "black", size = 6, fontface = "bold") + # Adjust hjust to position the asterisk +
geom_text(data = filter(predictors_of_outcome, term == "Study site 5"), 
            aes(label = "*"), hjust = 25, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

In [ ]:
ggsave(here("output","graph_lasso_oxytocin_cgi_24.png"))

### 12 weeks

#### ABC-SW Change

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
   abc_sw_change_12,

    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_duke,
    site_mt_sinai,
    site_mgh,
    site_vanderbilt,
    site_seattle,
    site_cornell,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
tidy(best_model)

lasso_12 <- best_model

predictors_of_outcome <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-SW Change at 12 weeks",
       x = "predictor",
       y = "importance")

In [ ]:
# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_duke" = "Study site 1",
    "site_mt_sinai" = "Study site 2",
    "site_mgh" = "Study site 3",
    "site_vanderbilt" = "Study site 4",
    "site_seattle" = "Study site 5",
    "site_cornell" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at 12 weeks\n in the oxytocin group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-mSW"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold") + # Adjust hjust to position the asterisk +
geom_text(data = filter(predictors_of_outcome, term == "SRS-2"), 
            aes(label = "*"), hjust = 1.65, color = "black", size = 6, fontface = "bold") +
geom_text(data = filter(predictors_of_outcome, term == "Study site 2"), 
            aes(label = "*"), hjust = 1.5, color = "black", size = 6, fontface = "bold")

In [ ]:
ggsave(here("output","graph_lasso_oxytocin_abc_12.png"))

#### CGI Improvement

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    cgi_improvement_wk_12,
    
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_duke,
    site_mt_sinai,
    site_mgh,
    site_vanderbilt,
    site_seattle,
    site_cornell,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 12 weeks",
       x = "predictor",
       y = "importance")


In [ ]:
# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_duke" = "Study site 1",
    "site_mt_sinai" = "Study site 2",
    "site_mgh" = "Study site 3",
    "site_vanderbilt" = "Study site 4",
    "site_seattle" = "Study site 5",
    "site_cornell" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-mSW",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ")

# Replace terms with new names in the data
predictors_of_cgi_improvement <- predictors_of_cgi_improvement |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms


In [ ]:
ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI-I at 12 weeks\n in the oxytocin group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Prescribed stimulant"), 
            aes(label = "*"), hjust = -5.5, color = "black", size = 6, fontface = "bold")

In [ ]:
ggsave(here("output","graph_lasso_oxytocin_cgi_12.png"))

# Sensitivity analyses

## No IQ

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    abc_sw_change_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    srs_raw_total_baseline,
    abc_sw_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) |> bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe |> prep(data = df_test) |> bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_placebo_abc_24 <- best_model

In [ ]:
tidy(lasso_placebo_abc_24) |> arrange(desc(estimate)) |> filter(estimate!=0)

In [ ]:
model = lm(abc_sw_change_24 ~ 
    abc_sw_baseline +
    site_id_1 +
    med_anticonvulsant +
    med_antipsychotic +
           race_black +
           med_gi +
           srs_raw_total_baseline +
           site_id_3 +
           race_more_than_one +
           age_months +
           med_antidepressant +
           male_sex +
           site_id_6 +
           latin_ethnicity +
           baseline_bmi_z_score +
           med_alpha_agonist,
    data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)
output

In [ ]:
df_placebo |> nrow()

In [ ]:
df |> nrow()

In [ ]:
df |> filter(!is.na(baseline_iq )) |> nrow()

In [ ]:
df |> skim()

## Placebo group

### 24 weeks

#### ABC-mSW Change: lasso

In [ ]:
df_placebo |> colnames()

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    abc_sw_change_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) |> bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe |> prep(data = df_test) |> bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_placebo_abc_24_sensitivity <- best_model

In [ ]:
tidy(lasso_placebo_abc_24_sensitivity) |> arrange(desc(estimate)) |> filter(estimate!=0)

In [ ]:
tidy(lasso_placebo_abc_24_sensitivity)

predictors_of_outcome <- tidy(lasso_placebo_abc_24_sensitivity) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-modified Social Withdrawal",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ",
    "abc_irritability_baseline" = "Baseline ABC-Irritability",
    "vineland_socialization_standard_baseline" = "Baseline VABS-II Socialization",
    "vineland_communication_standard_baseline" = "Baseline VABS-II Communication",
    "caregiver_strain_subjective_internal_baseline" = "Baseline CSQ Subjective Internal",
    "caregiver_strain_subjective_external_baseline" = "Baseline CSQ Subjective External",
    "caregiver_strain_objective_baseline" = "Baseline CSQ Objective",
    "abc_stereotypy_baseline" = "Baseline ABC-Stereotypy",
    "abc_speech_baseline" = "Baseline ABC-Speech",
    "abc_lethargy_baseline" = "Baseline ABC-modified Lethargy",
    "abc_hyperactivity_baseline" = "Baseline ABC-Hyperactivity"
)

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at\n 24 weeks in the placebo group\n retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-modified Social Withdrawal"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

#### ABC-mSW Change: unpenalized regression

In [ ]:
model = lm(abc_sw_change_24 ~ 
    abc_irritability_baseline +
    abc_sw_baseline +
    site_id_1 +
    med_alpha_agonist,
    data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)
output

#### CGI-I: lasso

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    cgi_improvement_wk_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

In [ ]:
predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-modified Social Withdrawal",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ",
    "abc_irritability_baseline" = "Baseline ABC-Irritability",
    "vineland_socialization_standard_baseline" = "Baseline VABS-II Socialization",
    "vineland_communication_standard_baseline" = "Baseline VABS-II Communication",
    "caregiver_strain_subjective_internal_baseline" = "Baseline CSQ Subjective Internal",
    "caregiver_strain_subjective_external_baseline" = "Baseline CSQ Subjective External",
    "caregiver_strain_objective_baseline" = "Baseline CSQ Objective",
    "abc_stereotypy_baseline" = "Baseline ABC-Stereotypy",
    "abc_speech_baseline" = "Baseline ABC-Speech",
    "abc_lethargy_baseline" = "Baseline ABC-modified Lethargy",
    "abc_hyperactivity_baseline" = "Baseline ABC-Hyperactivity"
)

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_cgi_improvement |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI-I change at\n 24 weeks in the placebo group\n retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Study site 3"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold") + # Adjust hjust to position the asterisk
geom_text(data = filter(predictors_of_outcome, term == "Baseline CSQ Subjective External"), 
            aes(label = "*"), hjust = -3, color = "black", size = 6, fontface = "bold")

In [ ]:
ggsave("sensitivity_2.png")

#### CGI-I: unpenalized regression

In [ ]:
model = lm(cgi_improvement_wk_24 ~
    site_id_3 +
    caregiver_strain_subjective_external_baseline +
    srs_raw_total_baseline +
    abc_stereotypy_baseline,
        data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)

### 12 weeks

#### ABC-mSW Change: lasso

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    abc_sw_change_12,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) |> bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe |> prep(data = df_test) |> bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_placebo_abc_12_sensitivity <- best_model

In [ ]:
tidy(lasso_placebo_abc_12_sensitivity)

predictors_of_outcome <- tidy(lasso_placebo_abc_12_sensitivity) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-modified Social Withdrawal",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ",
    "abc_irritability_baseline" = "Baseline ABC-Irritability",
    "vineland_socialization_standard_baseline" = "Baseline VABS-II Socialization",
    "vineland_communication_standard_baseline" = "Baseline VABS-II Communication",
    "caregiver_strain_subjective_internal_baseline" = "Baseline CSQ Subjective Internal",
    "caregiver_strain_subjective_external_baseline" = "Baseline CSQ Subjective External",
    "caregiver_strain_objective_baseline" = "Baseline CSQ Objective",
    "abc_stereotypy_baseline" = "Baseline ABC-Stereotypy",
    "abc_speech_baseline" = "Baseline ABC-Speech",
    "abc_lethargy_baseline" = "Baseline ABC-modified Lethargy",
    "abc_hyperactivity_baseline" = "Baseline ABC-Hyperactivity")

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at\n 12 weeks in the placebo group\n retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-modified Lethargy"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

#### ABC-mSW Change: unpenalized regression

In [ ]:
model = lm(abc_sw_change_12 ~ 
    abc_irritability_baseline +
    abc_sw_baseline,
    data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)
output

In [ ]:
ggsave("sensitivity_3.png")

#### CGI-I: lasso

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    cgi_improvement_wk_12,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12_abc <- best_model

In [ ]:
set.seed(1)

df_for_model <- df_placebo |> select(c(
    cgi_improvement_wk_12,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12_abc <- best_model

In [ ]:
hist(df$baseline_iq)

In [ ]:
predictors_of_cgi_improvement <- tidy(lasso_12_abc) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-modified Social Withdrawal",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ",
    "abc_irritability_baseline" = "Baseline ABC-Irritability",
    "vineland_socialization_standard_baseline" = "Baseline VABS-II Socialization",
    "vineland_communication_standard_baseline" = "Baseline VABS-II Communication",
    "caregiver_strain_subjective_internal_baseline" = "Baseline CSQ Subjective Internal",
    "caregiver_strain_subjective_external_baseline" = "Baseline CSQ Subjective External",
    "caregiver_strain_objective_baseline" = "Baseline CSQ Objective",
    "abc_stereotypy_baseline" = "Baseline ABC-Stereotypy",
    "abc_speech_baseline" = "Baseline ABC-Speech",
    "abc_lethargy_baseline" = "Baseline ABC-modified Lethargy",
    "abc_hyperactivity_baseline" = "Baseline ABC-Hyperactivity"
)

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_cgi_improvement |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI-I change at\n 12 weeks in the placebo group\n retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Age"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold") + # Adjust hjust to position the asterisk
geom_text(data = filter(predictors_of_outcome, term == "Study site 1"), 
            aes(label = "*"), hjust = -3, color = "black", size = 6, fontface = "bold") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline CSQ Subjective External"), 
            aes(label = "*"), hjust = -3, color = "black", size = 6, fontface = "bold")

In [ ]:
predictors_of_cgi_improvement

In [ ]:
model = lm(cgi_improvement_wk_12 ~
age_months +
site_id_6	+ 
latin_ethnicity	+
med_sleep +
med_stimulant +
site_id_4 +
med_antipsychotic +
abc_speech_baseline	+
race_asian +
site_id_1 +
med_gi +
abc_sw_baseline,
        data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)

In [ ]:
predictors_of_cgi_improvement |> arrange(desc(estimate)) |> filter(estimate!=0)

In [ ]:
predictors_of_cgi_improvement |> arrange(desc(estimate)) |> filter(estimate!=0)

In [ ]:
ggsave("sensitivity_4.png")

#### CGI-I: unpenalized regression

In [ ]:
model = lm(cgi_improvement_wk_12 ~
    age_months +
caregiver_strain_subjective_internal_baseline +
site_id_6 +
caregiver_strain_subjective_external_baseline +
abc_sw_baseline +
           med_stimulant +
           med_gi +
           site_id_1 +
           site_id_4 +
           abc_speech_baseline,
        data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)

## Treatment group

### 24 weeks

#### ABC-mSW Change: lasso

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    abc_sw_change_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) |> bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe |> prep(data = df_test) |> bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_tx_abc_24_sensitivity <- best_model

tidy(lasso_tx_abc_24_sensitivity)

predictors_of_outcome <- tidy(lasso_tx_abc_24_sensitivity) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-modified Social Withdrawal",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ",
    "abc_irritability_baseline" = "Baseline ABC-Irritability",
    "vineland_socialization_standard_baseline" = "Baseline VABS-II Socialization",
    "vineland_communication_standard_baseline" = "Baseline VABS-II Communication",
    "caregiver_strain_subjective_internal_baseline" = "Baseline CSQ Subjective Internal",
    "caregiver_strain_subjective_external_baseline" = "Baseline CSQ Subjective External",
    "caregiver_strain_objective_baseline" = "Baseline CSQ Objective",
    "abc_stereotypy_baseline" = "Baseline ABC-Stereotypy",
    "abc_speech_baseline" = "Baseline ABC-Speech",
    "abc_lethargy_baseline" = "Baseline ABC-modified Lethargy",
    "abc_hyperactivity_baseline" = "Baseline ABC-Hyperactivity"
)

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at 24\n weeks in the treatment\n group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-modified Lethargy"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

In [ ]:
ggsave("sensitivity_tx_1.png")

#### ABC-mSW Change: unpenalized regression

In [ ]:
model = lm(abc_sw_change_24 ~ 
    abc_sw_baseline,
    data = df_tx)
summary(model)
output <- tidy(model, conf.int=TRUE)
output

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    abc_sw_change_12,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(abc_sw_change_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.01)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) |> bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = abc_sw_change_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe |> prep(data = df_test) |> bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$abc_sw_change_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

In [ ]:
lasso_tx_abc_24_sensitivity <- best_model

tidy(lasso_tx_abc_24_sensitivity)

predictors_of_outcome <- tidy(lasso_tx_abc_24_sensitivity) |>
    filter(term!="(Intercept)")

# Create a named vector where names are the original terms, and values are the new names
new_names <- c(
  "age_months" = "Age",
  "male_sex" = "Male sex",
  "latin_ethnicity" = "Latine ethnicity",
  "race_white" = "White race",
  "race_black" = "Black race",
    "race_asian" = "Asian race",
    "race_more_than_one" = "More than one race",
    "site_id_1" = "Study site 1",
    "site_id_2" = "Study site 2",
    "site_id_3" = "Study site 3",
    "site_id_4" = "Study site 4",
    "site_id_5" = "Study site 5",
    "site_id_6" = "Study site 6",
    "cgi_severity_baseline" = "CGI-S",
    "srs_raw_total_baseline" = "SRS-2",
    "abc_sw_baseline" = "Baseline ABC-modified Social Withdrawal",
    "baseline_bmi_z_score" = "BMI z-score",
    "med_alpha_agonist" = "Prescribed alpha agonist",
    "med_anxiolytic" = "Prescribed anxiolytic",
    "med_anticonvulsant" = "Prescribed anticonvulsant",
    "med_antipsychotic" = "Prescribed antipsychotic",
    "med_stimulant" = "Prescribed stimulant",
    "med_sleep" = "Prescribed sleep aid",
    "med_gi" = "Prescribed GI medication",
    "med_antidepressant" = "Prescribed antidepressant",
    "baseline_iq" = "SB5 Abbreviated IQ",
    "abc_irritability_baseline" = "Baseline ABC-Irritability",
    "vineland_socialization_standard_baseline" = "Baseline VABS-II Socialization",
    "vineland_communication_standard_baseline" = "Baseline VABS-II Communication",
    "caregiver_strain_subjective_internal_baseline" = "Baseline CSQ Subjective Internal",
    "caregiver_strain_subjective_external_baseline" = "Baseline CSQ Subjective External",
    "caregiver_strain_objective_baseline" = "Baseline CSQ Objective",
    "abc_stereotypy_baseline" = "Baseline ABC-Stereotypy",
    "abc_speech_baseline" = "Baseline ABC-Speech",
    "abc_lethargy_baseline" = "Baseline ABC-modified Lethargy",
    "abc_hyperactivity_baseline" = "Baseline ABC-Hyperactivity"
)

# Replace terms with new names in the data
predictors_of_outcome <- predictors_of_outcome |> 
  mutate(term = recode(term, !!!new_names))  # Apply new names to terms

# Now plot with reordered terms
ggplot(predictors_of_outcome, aes(x = reorder(term, estimate), y = estimate, fill = estimate > 0)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-mSW change at 24\n weeks in the treatment\n group retained by lasso",
       x = "Predictor",
       y = "Lasso coefficient") +
  theme(legend.position = "none") +
geom_text(data = filter(predictors_of_outcome, term == "Baseline ABC-modified Lethargy"), 
            aes(label = "*"), hjust = -0.3, color = "black", size = 6, fontface = "bold")  # Adjust hjust to position the asterisk

In [ ]:
tidy(lasso_tx_abc_24_sensitivity) |> arrange(desc(estimate)) |> filter(estimate!=0)

In [ ]:
model = lm(abc_sw_change_12 ~ 
    abc_sw_baseline +
    caregiver_strain_subjective_external_baseline +
           site_id_6 +
           race_more_than_one +
           cgi_severity_baseline +
med_gi +
           baseline_iq +
           vineland_communication_standard_baseline +
           race_white +
           site_id_5 +
           med_anticonvulsant +
           latin_ethnicity +
           srs_raw_total_baseline +
           site_id_2,
    data = df_tx)
summary(model)
output <- tidy(model, conf.int=TRUE)
output

In [ ]:
tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 24 weeks",
       x = "predictor",
       y = "importance")


#### CGI-I: lasso

In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    cgi_improvement_wk_24,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_lethargy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_24 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_24 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_24)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 24 weeks",
       x = "predictor",
       y = "importance")


In [ ]:
set.seed(1)

df_for_model <- df_tx |> select(c(
    cgi_improvement_wk_12,
    age_months,
    male_sex,
    latin_ethnicity,
    
    race_white,
    race_black,
    race_asian,
    race_more_than_one,
    
    site_id_1,
    site_id_2,
    site_id_3,
    site_id_4,
    site_id_5,
    site_id_6,
    
    cgi_severity_baseline,
    
    srs_raw_total_baseline,
    
    abc_sw_baseline,
    abc_speech_baseline,
    abc_hyperactivity_baseline,
    abc_stereotypy_baseline,
    abc_lethargy_baseline,
    abc_irritability_baseline,

    vineland_socialization_standard_baseline,
    vineland_communication_standard_baseline,

    caregiver_strain_objective_baseline,
    caregiver_strain_subjective_internal_baseline,
    caregiver_strain_subjective_external_baseline,
    
    baseline_bmi_z_score,
    
    med_alpha_agonist,
    med_anxiolytic,
    med_anticonvulsant,
    med_antipsychotic,
    med_stimulant,
    med_sleep,
    med_gi,
    med_antidepressant,

    baseline_iq))

set.seed(1)
split <- initial_split(df_for_model, prop = 0.7)
df_train <- training(split)
df_test <- testing(split)

recipe <- recipe(cgi_improvement_wk_12 ~ ., data = df_train) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_unknown(all_nominal_predictors()) |>
    step_dummy(all_nominal_predictors()) |>
    step_zv()

lambda_values <- seq(0.1, 4.0, by = 0.1)

best_model <- NULL
best_rmse <- Inf

for (lambda in lambda_values) {
  # define a model using this lambda
    lasso_spec <- linear_reg(penalty = lambda, engine = "glmnet", mixture = 1) |>
        set_engine("glmnet") |>
        set_mode("regression")

  # preprocess and fit training data
    preprocessed_train <- recipe |> prep(data = df_train) %>% bake(new_data = df_train)
    lasso_model <- fit(lasso_spec, data = preprocessed_train, formula = cgi_improvement_wk_12 ~ .)

  # preprocess and fit testing data
    preprocessed_test <- recipe %>% prep(data = df_test) %>% bake(new_data = df_test)

  # make predictions
    predictions <- predict(lasso_model, new_data = preprocessed_test)

  # calculate rmse from predictions
    rmse <- sqrt(mean((predictions$.pred - df_test$cgi_improvement_wk_12)^2, na.rm=TRUE))
    
    if(is.na(rmse)) {
        rmse <- 0
    }

  # is the current model the best so far?
    if (rmse < best_rmse) {
        best_rmse <- rmse
        best_model <- lasso_model
    }
}

summary(predict(best_model, new_data = preprocessed_test))

tidy(best_model)

lasso_12 <- best_model

predictors_of_cgi_improvement <- tidy(lasso_12) |>
    filter(term!="(Intercept)")

ggplot(predictors_of_cgi_improvement, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 12 weeks",
       x = "predictor",
       y = "importance")


#### CGI-I: unpenalized regression

In [ ]:
model = lm(cgi_improvement_wk_24 ~
    med_antipsychotic +
    male_sex +
    site_id_6 +
    site_id_5,
        data = df_tx)
summary(model)
output <- tidy(model, conf.int=TRUE)
output

# Unpenalized regressions

## Placebo group

### Baseline ABC-mSW as predictor of ABC-mSW change at 12 and 24 weeks

In [ ]:
range(df_placebo$abc_sw_baseline, na.rm=TRUE)

In [ ]:
range(df_placebo$abc_sw_change_24, na.rm=TRUE)

In [ ]:
range(df_placebo$abc_sw_change_12, na.rm=TRUE)

In [ ]:
ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "Baseline ABC-mSW vs. change at 24 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 24 weeks") +
    theme_minimal()
ggsave(here("output","graph_unpenalized_placebo_24.png"))

ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_12)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "Baseline ABC-mSW vs. change at 12 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 12 weeks") +
    theme_minimal()
ggsave(here("output","graph_unpenalized_placebo_12.png"))

In [ ]:
ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_24)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  xlim(0,35) +  # Set x axis limits
  ylim(-15, 25) +  # Set y axis limits
  labs(title = "Baseline ABC-mSW vs. change at 24 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 24 weeks") +
    theme_minimal()

ggplot(df_placebo, aes(x = abc_sw_baseline, y = abc_sw_change_12)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "cornflowerblue") + # unpenalized regression line
  labs(title = "Baseline ABC-mSW vs. change at 12 weeks in the placebo group",
       x = "Baseline ABC-mSW",
       y = "Change in ABC-mSW at 12 weeks") +
    theme_minimal()


### 24 weeks

#### ABC-SW Change

In [ ]:
library(kableExtra)

In [ ]:
model = lm(abc_sw_change_24 ~ 
           abc_sw_baseline +
           site_duke + med_alpha_agonist,
        data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)

In [ ]:
output

In [ ]:
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "<i>SE</i>", "t", "95% CI", "", "p"),
       caption = "Predictors of ABC-mSW change at 24 weeks in the placebo group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

In [ ]:
model = lm(cgi_improvement_wk_24 ~ cgi_severity_baseline,
        data = df_placebo)
summary(model)
output <- tidy(model, conf.int=TRUE)

### 12 weeks

#### ABC-SW change

In [ ]:
model = lm(abc_sw_change_12 ~ 
           abc_sw_baseline +
srs_raw_total_baseline +
race_black +
site_duke +
med_antidepressant +
site_cornell,
        data = df_placebo)
summary(model)

In [ ]:
output <- tidy(model, conf.int=TRUE)

In [ ]:
output

In [ ]:
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "SE", "t", "95% CI", "", "p"),
       caption = "Predictors of ABC-mSW change at 24 weeks in the placebo group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

In [ ]:
model = lm(abc_sw_change_12 ~ 
           srs_raw_total_baseline,
        data = df_placebo)
summary(model)
tidy(model)

In [ ]:
model = lm(abc_sw_change_12 ~ 
           med_antidepressant,
        data = df_placebo)
summary(model)
tidy(model)

In [ ]:
model = lm(abc_sw_change_12 ~ 
           race +
           abc_sw_baseline +
           srs_raw_total_baseline +
           med_antidepressant,
        data = df_placebo)
summary(model)
tidy(model)

In [ ]:
model = lm(abc_sw_change_24 ~ 
           abc_sw_baseline +
           med_alpha_agonist,
        data = df_placebo)
summary(model)
tidy(model)

In [ ]:
model = lm(abc_sw_change_12 ~ 
           race +
           abc_sw_baseline +
           srs_raw_total_baseline +
           med_antidepressant +
           site_id,
        data = df_placebo)
summary(model)
tidy(model)

In [ ]:
predictors <- tidy(model) |>
    filter(term!="(Intercept)") 

ggplot(predictors, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-SW change at 12 weeks",
       x = "predictor",
       y = "estimate")


#### CGI-I Improvement

In [ ]:
model = lm(cgi_improvement_wk_12 ~ 
race_black +
site_seattle +
age_months +
site_cornell +
baseline_iq +
abc_sw_baseline +
race_asian +
med_stimulant +
male_sex +
med_gi +
site_duke +
site_vanderbilt,
        data = df_placebo)
summary(model)

In [ ]:
output <- tidy(model, conf.int=TRUE)

In [ ]:
output

In [ ]:
output <- tidy(model, conf.int=TRUE)
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "SE", "t", "95% CI", "", "p"),
       caption = "Predictors of CGI-I at 12 weeks in the placebo group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

#### 12 weeks: ABC-SW Change

In [ ]:
model = lm(abc_sw_change_12 ~ 
           race +
           srs_raw_total_baseline +
           abc_sw_baseline +
           site_id,
        data = df_placebo)
summary(model)
tidy(model)

#### 24 weeks: CGI Improvement

In [ ]:
model = lm(cgi_improvement_wk_24 ~ 
           age_months +
           male_sex +
           latin_ethnicity +
           race +
           site_id +
           cgi_severity_baseline +
           srs_raw_total_baseline +
           abc_sw_baseline +
           baseline_iq,
        data = df_placebo)
summary(model)

In [ ]:
predictors <- tidy(model) |>
    filter(term!="(Intercept)") 

ggplot(predictors, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of CGI Improvement at 24 weeks",
       x = "predictor",
       y = "estimate")


In [ ]:
predictors <- tidy(model) |>
    filter(term!="(Intercept)") 

ggplot(predictors, aes(x = reorder(term, estimate), y = estimate)) +
  geom_bar(stat = "identity") +
  coord_flip() +  # Flip the coordinates for better readability
  theme_minimal() +
  labs(title = "Predictors of ABC-SW change at 24 weeks",
       x = "predictor",
       y = "estimate")


In [ ]:
vif(model)

## Treatment group

### 24 weeks

#### ABC-SW Change

In [ ]:
model = lm(abc_sw_change_24 ~ 
abc_sw_baseline +
age_months +
male_sex +
site_vanderbilt +
race_more_than_one +
race_black +
site_duke +
med_alpha_agonist +
baseline_bmi_z_score +
baseline_iq +
med_stimulant +
med_antidepressant +
med_antipsychotic +
med_sleep +
site_mgh  +
latin_ethnicity +
site_mt_sinai +
srs_raw_total_baseline,     
        data = df_tx)
summary(model)
tidy(model)

In [ ]:
output <- tidy(model, conf.int=TRUE)
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "SE", "t", "95% CI", "", "p"),
       caption = "Predictors of ABC-mSW change at 24 weeks in the oxytocin group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

#### CGI-I

In [ ]:
model = lm(cgi_improvement_wk_24 ~ 
male_sex +
med_antipsychotic +
site_cornell +
latin_ethnicity +
age_months +
site_mgh +
abc_sw_baseline +
med_sleep +
site_seattle,
        data = df_tx)
summary(model)
tidy(model)

output <- tidy(model, conf.int=TRUE)

In [ ]:
tidy(model, conf.int=TRUE)

In [ ]:
output <- tidy(model, conf.int=TRUE)
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "SE", "t", "95% CI", "", "p"),
       caption = "Predictors of CGI-I at 24 weeks in the oxytocin group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

### 12 weeks

#### ABC-mSW change

In [ ]:
model = lm(abc_sw_change_12 ~ 
abc_sw_baseline +
site_cornell +
cgi_severity_baseline +
race_more_than_one +
baseline_iq +
med_sleep +
latin_ethnicity +
srs_raw_total_baseline +
race_white +
site_mt_sinai,
        data = df_tx)
summary(model)
tidy(model)

In [ ]:
output <- tidy(model, conf.int=TRUE)
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "SE", "t", "95% CI", "", "p"),
       caption = "Predictors of ABC-mSW change at 12 weeks in the oxytocin group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

#### CGI-I

In [ ]:
model = lm(cgi_improvement_wk_12 ~ 
male_sex +
site_mgh +
med_stimulant +
baseline_iq + 
site_mt_sinai +
site_seattle +
race_more_than_one +
cgi_severity_baseline +
site_vanderbilt,
        data = df_tx)
summary(model)
tidy(model)

In [ ]:
output <- tidy(model, conf.int=TRUE)
output <- output |> select(-p.value,p.value)
output %>%
  kable("html", digits = 3, col.names = c("Term", "Coefficient", "SE", "t", "95% CI", "", "p"),
       caption = "Predictors of CGI-I at 12 weeks in the oxytocin group") %>%
  kable_styling(bootstrap_options = c("basic"),
                full_width = F, 
                position = "left")

#### 24 weeks: ABC-SW Change

- Variables significant in the LASSO - do individual univariate regressions

In [ ]:
model = lm(abc_sw_change_24 ~
           abc_sw_baseline +
male_sex +
med_gi +
age_months +
med_stimulant +
cgi_severity_baseline +
med_alpha_agonist +
latin_ethnicity +
srs_raw_total_baseline,
        data = df_tx)
summary(model)
tidy(model)

# Sex interaction

In [ ]:
anova_model <- aov(cgi_improvement_wk_24 ~ male_sex * tx_group, data = df)
summary(anova_model)
tidy(anova_model)

In [ ]:
subset <- df |> select(tx_group, male_sex, cgi_improvement_wk_24)

In [ ]:
subset <- na.omit(subset)

In [ ]:
interaction.plot(subset$tx_group, subset$male_sex, subset$cgi_improvement_wk_24,
                 main = "Interaction between sex and treatment group",
                 xlab = "Treatment group", ylab = "CGI improvement at week 24",
                 trace.label = "Male sex")

In [ ]:
ggplot(df, aes(x = tx_group, y = cgi_improvement_wk_24, color = male_sex)) +
  geom_point() +
    geom_jitter() +
  geom_smooth(method = "lm", aes(group = interaction(male_sex, tx_group)), se = FALSE) +
  labs(x = "Treatment", y = "Social Function", title = "Individual Data Points by Sex and Treatment") +
  theme_minimal()

In [ ]:
model <- lm(cgi_improvement_wk_24 ~ male_sex * tx_group, data = df)

In [ ]:
summary(model)
tidy(model, conf.int = TRUE)

In [ ]:
df_viz <- df

In [ ]:
df_viz$male_sex <- as.factor(df_viz$male_sex)

In [ ]:
df_viz$tx_group <- as.factor(df_viz$tx_group)

In [ ]:
ggplot(df, aes(x = tx_group, y = cgi_improvement_wk_24, fill = tx_group)) +
  geom_bar(stat = "summary", fun = "mean", position = "dodge") +
  facet_grid(~ male_sex) +
  labs(x = "Treatment", y = "Social Function", title = "Interaction of Sex and Treatment") +
  theme_minimal()

In [ ]:
ggplot(df_viz, aes(x = tx_group, y = cgi_improvement_wk_24, color = male_sex)) +
  geom_jitter(width = 0.3, height = 0.3, alpha = 0.5, size = 2) +
  stat_summary(fun = mean, geom = "line", aes(group = male_sex), linetype = "solid", size = 2) +
  labs(title = "Interaction between sex and treatment group",
       x = "Treatment group",
       y = "CGI improvement at week 24",
       color = "Sex") +
  theme(legend.position = "top")

In [ ]:
library(ggplot2)
ggplot(df_viz, aes(x = tx_group, y = cgi_improvement_wk_24, color = factor(male_sex))) +
  geom_point(alpha = 0.5) +
    geom_jitter() +
#  geom_smooth(method = "lm", aes(group = interaction(male_sex, tx_group)), se = FALSE) +
labs(
    title = "Interaction between sex and treatment group on CGI improvement",
    x = "Treatment group",
    y = "CGI improvement at week 24",
    color = "Male sex"
  ) 


In [ ]:
df_viz <- df_viz[!is.na(df$cgi_improvement_wk_24), ]


In [ ]:
ggplot(df, aes(x = cgi_severity_baseline, y = abc_sw_change_12)) +
  geom_point(alpha = 0.5) +  # Scatter plot points
    geom_smooth(method = "lm", se = FALSE, color = "lightcoral") + # unpenalized regression line
  labs(title = "ABC-SW: baseline vs change at 12 weeks (unpenalized regression)",
       x = "Baseline ABC-SW score",
       y = "Change in ABC-SW score at 12 weeks")

In [ ]:
# Scatter plot with regression lines
ggplot(df_viz, aes(x = cgi_improvement_wk_24, y = tx_group, color = as.factor(male_sex))) +
#  geom_point(position = position_jitter(width = 0.1), alpha = 0.5) +  # Jitter for visibility
  geom_smooth(method = "lm", aes(group = interaction(male_sex, tx_group)), se = FALSE) +
  labs(
    title = "Interaction between sex and treatment group with CGI Improvement",
    x = "Treatment group",
    y = "CGI improvement at week 24",
    color = "Male sex"
  ) 

# Etc

## t-tests & ANOVAs

In [ ]:
t.test(abc_sw_change_12 ~ latin_ethnicity, data = df_placebo) |> tidy()

In [ ]:
t.test(abc_sw_change_24 ~ latin_ethnicity, data = df_placebo) |> tidy()

In [ ]:
t.test(abc_sw_change_12 ~ male_sex, data = df_placebo) |> tidy()

In [ ]:
t.test(abc_sw_change_24 ~ male_sex, data = df_placebo) |> tidy()

In [ ]:
t.test(abc_sw_change_24 ~ med_nonstimulant_adhd, data = df_placebo) |> tidy()

In [ ]:
aov(abc_sw_change_12 ~ site_id, data = df_placebo) |> tidy()

In [ ]:
aov(abc_sw_change_24 ~ site_id, data = df_placebo) |> tidy()

In [ ]:
aov(abc_sw_change_12 ~ race, data = df_placebo) |> tidy()

In [ ]:
aov(abc_sw_change_24 ~ race, data = df_placebo) |> tidy()